In [ ]:
"""

This script will analyze the output from transcript quantification with espresso and identify AS events
that overlap with genes with allele-specific splicing orders

"""

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import itertools
from tqdm import tqdm
from statsmodels.stats.multitest import multipletests

In [2]:
# Get espresso results
esp_df = pd.read_table("/path/to/results_files/LCL_espresso_counts.tsv")

# Get gene_ID to gene_name correspondence
ann_df = pd.read_table("/path/to/Ensembl_geneID2name_hsapiens_gene_ensembl.txt")

# Merge espresso results with gene_ID/gene_name
esp_df = esp_df.merge(ann_df[['ensembl_gene_id','external_gene_name']].drop_duplicates(), left_on=['gene_ID'], right_on=['ensembl_gene_id'])

In [17]:
# Reformat espresso results
esp_df_m = esp_df.melt(id_vars=['transcript_ID','transcript_name','gene_ID','ensembl_gene_id','external_gene_name'],
           var_name = 'allele_tmp', value_name = 'count')
esp_df_m['cell_line'] = esp_df_m['allele_tmp'].str.split("_").str[0]
esp_df_m['allele'] = esp_df_m['allele_tmp'].str.split("\\.").str[1]
esp_df_grp = pd.DataFrame(esp_df_m.groupby(['transcript_ID','gene_ID','external_gene_name','cell_line','allele'])['count'].sum()).reset_index()
esp_df_piv = esp_df_m.pivot_table(index=['transcript_ID','gene_ID','external_gene_name','cell_line'],columns='allele',values='count').reset_index()

In [3]:
# Get genes for which we computed splicing order
order_df = pd.read_table("/path/to/results_files/LCLs_splicing_order_merged_samples_chi_square_vs_distance_results.txt")
order_sig = order_df[(order_df['d']>0.379) & ((order_df['level1']<0.05) | (order_df['level2']<0.05))].reset_index(drop=True)
order_genes = order_sig[['gene_name','cell_line']].drop_duplicates().reset_index(drop=True)

In [21]:
# Function to test for statistical difference in isoform abundance between alleles
def compare_isos(gene_name):
    
    t = 10
    results_list = []
    
    transcripts = esp_df_piv[esp_df_piv['external_gene_name']==gene_name]['transcript_ID'].drop_duplicates().tolist()
    
    # get all transcript combinations
    for combo in itertools.combinations(transcripts, 2):
        t1 = combo[0]
        t2 = combo[1]
    
        # retrieve counts for t1 and t2
        counts_t1 = esp_df_piv[esp_df_piv['transcript_ID']==t1].reset_index(drop=True)
        counts_t2 = esp_df_piv[esp_df_piv['transcript_ID']==t2].reset_index(drop=True)
    
        # filter for cell lines that have a minimum reads on at least one allele
        t1_filt = counts_t1[(counts_t1['maternal']>=t) | (counts_t1['paternal']>=t)].reset_index(drop=True)
        t2_filt = counts_t2[(counts_t2['maternal']>=t) | (counts_t2['paternal']>=t)].reset_index(drop=True)
    
        # Identify cell lines that passed the filter for both transcripts
        t1_cell_lines = t1_filt['cell_line'].tolist()
        t2_cell_lines = t2_filt['cell_line'].tolist()
    
        good_cell_lines = [a for a in t1_cell_lines if a in t2_cell_lines]
    
        # test for statistically significant differences
        for ind in good_cell_lines:
            t1_p = int(t1_filt[t1_filt['cell_line']==ind]['paternal'])
            t2_p = int(t2_filt[t2_filt['cell_line']==ind]['paternal'])
            t1_m = int(t1_filt[t1_filt['cell_line']==ind]['maternal'])
            t2_m = int(t2_filt[t2_filt['cell_line']==ind]['maternal'])
        
            OR = stats.fisher_exact([[t1_p,t2_p],[t1_m,t2_m]])[0]
            pval = stats.fisher_exact([[t1_p,t2_p],[t1_m,t2_m]])[1]
            results_list.append([ind, gene_name, t1, t2, t1_p, t2_p, t1_m, t2_m, OR, pval])
    
    if len(results_list) > 0:
    
        results_df = pd.DataFrame(results_list)
        results_df.columns = ['cell_line','gene_name','transcript1','transcript2','count1_P','count2_P','count1_M','count2_M','odds_ratio','pvalue']
    
        return(results_df)
    

In [27]:
# Apply function defined above
df_list = []

for gene_name in tqdm(order_genes['gene_name'].drop_duplicates().tolist()):
#for gene_name in tqdm(gene_list_tmp):
    gene_df = compare_isos(gene_name)
    if gene_df is not None:
        df_list.append(gene_df)
    
final_df = pd.concat(df_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [1:29:32<00:00, 55.97s/it]


In [28]:
# Correct for multiple testing
final_df['FDR'] = multipletests(final_df['pvalue'], alpha=0.05, method='fdr_bh')[1]

In [29]:
# Filter for significant pvalue and odds ratio
final_df_sig = final_df[(final_df['FDR']<0.05) & ((final_df['odds_ratio']>2) | (final_df['odds_ratio']<0.5) | (final_df['odds_ratio']==0.00))].reset_index(drop=True)

In [32]:
# Save transcript pairs and use rMATS-long script classify_isoform_differences.py to determine the coordinates of AS events
final_df_sig.to_csv("/path/to/results_files/LCL_espresso_pairwise_transcript_comparison_2024-08-27.txt", sep="\t", header=True, index=False)

In [5]:
# Get pairwise annotation of alternative splicing differences between isoforms from rmats-long (with only significant pairs)
iso_df = pd.read_table("/path/to/results_files/LCL_isoform_differences.tsv")
esp_gene_names = esp_df[['transcript_ID','external_gene_name']].drop_duplicates().reset_index(drop=True)
iso_df = iso_df.merge(esp_gene_names, left_on=['transcript1'], right_on=['transcript_ID'])

# Filter out intron retention events (since we're analyzing chromatin RNA), alternative first/last exons and complex events
iso_df_sub = iso_df[~iso_df['event'].isin(['COMPLEX','RI','AFE','ALE'])].reset_index(drop=True)

In [17]:
# Identify isoforms that are in the same genes for which splicing order was measured in each cell line
order_regions = order_df[['gene_name','gene','analyzed_introns','cell_line','d','level1','level2']].drop_duplicates().reset_index(drop=True)

# Merge the stats results with the annotation of AS events
final_df_sig_ann = final_df_sig.merge(iso_df_sub, on=['transcript1','transcript2']).merge(order_regions, on=['cell_line','gene_name'])

In [11]:
# Get intron coordinates
hg38_intron_df = pd.read_table("/path/to/annotation_files/hg38_all_intron_features.txt")

In [12]:
# Define a function to determine whether the AS event overlaps with the intron group for which splicing order was
# computed

def AS_vs_order(gene, analyzed_introns, event_type, coord):
    
    window = 50
    
    # retrieve coordinates of analyzed introns for splicing order
    split_introns = [int(a) for a in analyzed_introns.split("_")]
    intron1 = split_introns[0]
    intron3 = split_introns[2]
    gr_start = int(hg38_intron_df[(hg38_intron_df['gene']==gene) & (hg38_intron_df['intron_pos']==intron1)]['start']) - window
    gr_end = int(hg38_intron_df[(hg38_intron_df['gene']==gene) & (hg38_intron_df['intron_pos']==intron3)]['end']) + window
    
    split_coord = coord.split(";")
    
    flag = "no_overlap"
    
    for event in split_coord:
        if flag == "overlap":
            pass
        split_event = event.split(":")
        event_start = int(split_event[1])
        event_end = int(split_event[2])
        strand = split_event[3]
        
        if event_type == "SE":
            if event_start > gr_start and event_end < gr_end:
                flag = "overlap"
            else:
                flag = "no_overlap"
                
        elif event_type == "A5SS":
            if strand == "+":
                if event_end > gr_start and event_end < gr_end:
                    flag = "overlap"
                else:
                    flag = "no_overlap"
            elif strand == "-":
                if event_start > gr_start and event_start < gr_end:
                    flag = "overlap"
                else:
                    flag = "no_overlap"
                    
        elif event_type == "A3SS":
            if strand == "+":
                if event_start > gr_start and event_start < gr_end:
                    flag = "overlap"
                else:
                    flag = "no_overlap"
            elif strand == "-":
                if event_end > gr_start and event_end < gr_end:
                    flag = "overlap"
                else:
                    flag = "no_overlap"
                    
        else:
            flag = "wrong_event_type"
            
    return(flag)
    

In [18]:
# Apply the function to each row
final_df_sig_ann['flag'] = final_df_sig_ann.apply(lambda row: AS_vs_order(row.gene, row.analyzed_introns, row.event, row.coordinates),axis=1)


In [ ]:
# Filter for AS events overlapping intron groups and that are not in HLA genes (those have some alignment artifacts
# that lead to "AS")
AS_in_order_genes = final_df_sig_ann[(final_df_sig_ann['flag']=='overlap') & (~final_df_sig_ann['gene_name'].str.contains("HLA"))].drop_duplicates(subset=['coordinates','cell_line'])
len(AS_in_order_genes)

In [44]:
# Get AS events in genes that showed allele-specific splicing order
AS_in_sig_order_genes = AS_in_order_genes[(AS_in_order_genes['d']>0.379) & ((AS_in_order_genes['level1']<0.05) | (AS_in_order_genes['level2']<0.05))]

In [45]:
AS_in_sig_order_genes

,cell_line,gene_name,transcript1,transcript2,count1_P,count2_P,count1_M,count2_M,odds_ratio,pvalue,...,event,coordinates,transcript_ID,external_gene_name,gene,analyzed_introns,d,level1,level2,flag
426,GM18510,FCER2,ENST00000346664,ESPRESSO:19:14393:5,242,0,223,10,inf,7.294634e-04,...,A5SS,19:7697527:7697589:-;19:7697401:7697589:-,ENST00000346664,FCER2,NM_001207019.2,4_3_2,0.759306,1.055303e-04,1.000000e+00,overlap
4324,GM18501,IFI44L,ENST00000370751,ESPRESSO:1:1108:25,14,14,105,33,0.314286,1.003321e-02,...,SE,1:78620944:78620971:+,ENST00000370751,IFI44L,NM_006820.3,1_2_3,0.448073,4.496140e-02,7.639776e-01,overlap
4326,GM18501,IFI44L,ENST00000370751,ESPRESSO:1:1108:51,14,32,105,0,0.000000,4.078529e-22,...,SE,1:78627906:78628393:+,ENST00000370751,IFI44L,NM_006820.3,1_2_3,0.448073,4.496140e-02,7.639776e-01,overlap
4505,GM18501,RPS2,ENST00000527871,ENST00000531065,27,0,1,14,inf,2.837672e-10,...,A5SS,16:1963653:1964365:-;16:1964276:1964365:-,ENST00000527871,RPS2,NM_002952.3,3_2_1,1.010153,2.461161e-05,1.000000e+00,overlap
4513,GM18853,RPS2,ENST00000527871,ENST00000531065,22,0,1,15,inf,1.034174e-09,...,A5SS,16:1963653:1964365:-;16:1964276:1964365:-,ENST00000527871,RPS2,NM_002952.3,3_2_1,1.196642,2.256142e-08,1.000000e+00,overlap
4521,GM19209,RPS2,ENST00000527871,ENST00000531065,22,0,0,19,inf,4.087260e-12,...,A5SS,16:1963653:1964365:-;16:1964276:1964365:-,ENST00000527871,RPS2,NM_002952.3,3_2_1,1.141695,9.420491e-06,1.000000e+00,overlap
4531,GM19152,SBF1,ESPRESSO:22:20218:32,ESPRESSO:22:20218:33,19,29,19,1,0.034483,2.639141e-05,...,SE,22:50457034:50457111:-,ESPRESSO:22:20218:32,SBF1,NM_002972.3,30_29_28,0.974978,3.178884e-01,3.428553e-12,overlap
